In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

получаем таблицу значений

In [2]:
df = pd.read_csv('winequality-white.csv', sep=';', decimal='.', na_values='NaN')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [3]:
#  разделяю предикоторы и отклики
X = df.iloc[:, :-1] 
y = df.iloc[:, 11]  

In [4]:
#  разделяю на обучающую и тестовую выборку

from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)  


In [5]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

model = GradientBoostingClassifier(random_state=42,
                                   # Число деревьев
                                   n_estimators=30,
                                   #  загрязнение измеряем “mse”, “mae” или “friedman_mse” (mse с улучшениями),  
                                   criterion='friedman_mse', 
                                   # Максимальная глубина каждого дерева
                                   #  критерий качества ‘deviance’ (кросс-энтропия) или ‘exponential’ (как в AdaBoost)
                                   #  ‘deviance’ для классификации с вероятностями на выходе
                                   loss='deviance', 
                                   # минимальное уменьшение загрязнения 
                                   min_impurity_decrease=0.0, 
                                   # Устарело
                                   min_impurity_split=None,
                                   # число узлов в дереве (сравни с RandomForest) !!!!!!!!!!!!!
                                   max_depth=5,
                                   # минимальное число наблюдений в узле потомке
                                   min_samples_leaf=5, 
                                   # минимальное число наблюдений в узле родителе
                                   min_samples_split=10,
                                   # Параметр, уменьшающий переобучение, являющемся весом отдельного дерева.
                                   # Рекомендуется выставлять небольшие значения из (0, 0.3].
                                   learning_rate=0.01
                                   # Есть и другие параметры, уменьшающие размер дерева,
                                   # такие же как у DecisionTreeClassifier
                                   )


In [6]:
model.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=5,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=5, min_samples_split=10,
              min_weight_fraction_leaf=0.0, n_estimators=30,
              n_iter_no_change=None, presort='auto', random_state=42,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [7]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           3       0.25      0.25      0.25         4
           4       0.42      0.10      0.16        80
           5       0.57      0.56      0.57       724
           6       0.54      0.73      0.62      1125
           7       0.50      0.21      0.29       430
           8       0.31      0.06      0.10        83
           9       0.00      0.00      0.00         3

   micro avg       0.54      0.54      0.54      2449
   macro avg       0.37      0.27      0.28      2449
weighted avg       0.53      0.54      0.51      2449



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
y_pred_train = model.predict(X_train)
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

           3       1.00      0.12      0.22        16
           4       0.88      0.18      0.30        83
           5       0.71      0.67      0.69       733
           6       0.59      0.84      0.70      1073
           7       0.81      0.32      0.46       450
           8       0.93      0.27      0.42        92
           9       0.00      0.00      0.00         2

   micro avg       0.65      0.65      0.65      2449
   macro avg       0.70      0.35      0.40      2449
weighted avg       0.69      0.65      0.62      2449



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
fi = pd.DataFrame({'features': X_train.columns, 'importance': model.feature_importances_})
fi.sort_values('importance', ascending=False).head(10)

,features,importance
10,alcohol,0.349934
1,volatile acidity,0.206513
5,free sulfur dioxide,0.086067
3,residual sugar,0.068268
7,density,0.061055
4,chlorides,0.051963
9,sulphates,0.044222
8,pH,0.036986
2,citric acid,0.036737
6,total sulfur dioxide,0.034921


Модель по самому большому классу 

In [10]:
df = pd.read_csv('winequality-white.csv', sep=';', decimal='.', na_values='NaN')

In [11]:
# новые предикторы
y_train[y_train != 6] = 0
y_test[y_test != 6] = 0

y_train[y_train == 6] = 1
y_test[y_test == 6] = 1


In [12]:
y_train.value_counts()

0    1376
1    1073
Name: quality, dtype: int64

In [13]:
model = GradientBoostingClassifier(random_state=42,
                                   # Число деревьев
                                   n_estimators=30,
                                   #  загрязнение измеряем “mse”, “mae” или “friedman_mse” (mse с улучшениями),  
                                   criterion='friedman_mse', 
                                   # Максимальная глубина каждого дерева
                                   #  критерий качества ‘deviance’ (кросс-энтропия) или ‘exponential’ (как в AdaBoost)
                                   #  ‘deviance’ для классификации с вероятностями на выходе
                                   loss='deviance', 
                                   # минимальное уменьшение загрязнения 
                                   min_impurity_decrease=0.0, 
                                   # Устарело
                                   min_impurity_split=None,
                                   # число узлов в дереве (сравни с RandomForest) !!!!!!!!!!!!!
                                   max_depth=5,
                                   # минимальное число наблюдений в узле потомке
                                   min_samples_leaf=5, 
                                   # минимальное число наблюдений в узле родителе
                                   min_samples_split=10,
                                   # Параметр, уменьшающий переобучение, являющемся весом отдельного дерева.
                                   # Рекомендуется выставлять небольшие значения из (0, 0.3].
                                   learning_rate=0.01
                                   # Есть и другие параметры, уменьшающие размер дерева,
                                   # такие же как у DecisionTreeClassifier
                                   )


In [14]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [15]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.55      0.99      0.71      1324
           1       0.83      0.03      0.06      1125

   micro avg       0.55      0.55      0.55      2449
   macro avg       0.69      0.51      0.38      2449
weighted avg       0.68      0.55      0.41      2449



In [16]:
y_pred_train = model.predict(X_train)
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

           0       0.57      1.00      0.73      1376
           1       0.98      0.05      0.10      1073

   micro avg       0.58      0.58      0.58      2449
   macro avg       0.78      0.52      0.41      2449
weighted avg       0.75      0.58      0.45      2449



и проведем калибровку

In [17]:
from sklearn.calibration import CalibratedClassifierCV

In [18]:
model_sigmoid = CalibratedClassifierCV(model, cv=2, method='sigmoid')
# method : ‘sigmoid’ or ‘isotonic’

In [19]:
# Calibrate probabilities
model_sigmoid.fit(X_train, y_train)

CalibratedClassifierCV(base_estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=5,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=5, min...      subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
            cv=2, method='sigmoid')

In [20]:
# View calibrated probabilities
model_sigmoid.predict_proba(X_test)[0:11, :]

array([[0.5587014 , 0.4412986 ],
       [0.50451038, 0.49548962],
       [0.69724642, 0.30275358],
       [0.71841368, 0.28158632],
       [0.63530622, 0.36469378],
       [0.505604  , 0.494396  ],
       [0.54807513, 0.45192487],
       [0.45615415, 0.54384585],
       [0.68774005, 0.31225995],
       [0.6289425 , 0.3710575 ],
       [0.505604  , 0.494396  ]])